<a href="https://colab.research.google.com/github/aashikrasool/Unsupervised-Image-Super-Resolution/blob/main/pretrained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import os

In [ ]:
#mount google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# assigning  data path 
dir_dataset = "/content/drive/MyDrive/Dataset/SR/HR"
files_img = [os.path.join(dir_dataset, x) for x in os.listdir(dir_dataset)]
     

In [ ]:
#to showing image from data set
from google.colab.patches import cv2_imshow

img = cv2.imread(files_img[8], cv2.IMREAD_UNCHANGED)
cv2_imshow(img)

In [ ]:
import numpy as np
def downsample(img_file, scale=0.3, plot= False):
    img = cv2.imread(img_file, cv2.IMREAD_UNCHANGED)
    img_small= cv2.resize(img,(0,0), fx=scale, fy=scale, interpolation=cv2.INTER_NEAREST)

    if plot: 
        img_small_resize= cv2.resize(img_small, (img.shape[0], img.shape[1]))
        cv2_imshow(np.hstack([img, img_small_resize]))
        return img, img_small
    
    return img, img_small

_, img_small = downsample(files_img[8], scale=0.4 , plot=True)

In [ ]:
!rm -rf pretrained_models
!wget https://github.com/Saafke/EDSR_Tensorflow/raw/master/models/EDSR_x4.pb -P pretrained_models -q
!wget https://github.com/fannymonori/TF-ESPCN/raw/master/export/ESPCN_x4.pb -P pretrained_models -q
!wget https://github.com/Saafke/FSRCNN_Tensorflow/raw/master/models/FSRCNN_x4.pb -P pretrained_models -q
!wget https://github.com/fannymonori/TF-LapSRN/raw/master/export/LapSRN_x4.pb -P pretrained_models -q

In [ ]:
dir_pretrained_models = 'pretrained_models'
os.listdir(dir_pretrained_models)

In [ ]:
# ''' Model upscale any image using opencv and external pretrained models. '''
def get_upscaled_images(img_small, filemodel_filepath, modelname, scale):

    model_pretrained = cv2.dnn_superres.DnnSuperResImpl_create()
    print("Reading model file {}".format(filemodel_filepath))
  
    # setting up the model initialization
    model_pretrained.readModel(filemodel_filepath)
    model_pretrained.setModel(modelname, scale)
  
    # prediction or upscaling
    img_upscaled = model_pretrained.upsample(img_small)
    return img_upscaled
img, img_small = downsample(files_img[8], scale=0.25)
print(img.shape, img_small.shape)
img_upscaled1 = get_upscaled_images(img_small, "pretrained_models/EDSR_x4.pb", "edsr", 4)
img_upscaled2 = get_upscaled_images(img_small, "pretrained_models/ESPCN_x4.pb", "espcn", 4)
img_upscaled3 = get_upscaled_images(img_small, "pretrained_models/FSRCNN_x4.pb", "fsrcnn", 4)
img_upscaled4 = get_upscaled_images(img_small, "pretrained_models/LapSRN_x4.pb", "lapsrn", 4)

print(img_upscaled1.shape, img_upscaled2.shape, img_upscaled3.shape, img_upscaled4.shape)

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

def plot_images(images, titles):
    fig = plt.figure(figsize=(20., 8.))
    grid = ImageGrid(fig, 111, nrows_ncols=(1, len(images)), axes_pad=0.1)

    i = 0
    for ax, img in zip(grid, images):
        ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        ax.set_title(titles[i])
        i += 1
    plt.show()

img_small_resize = cv2.resize(img_small, (img.shape[0], img.shape[1]))

titles = ["original", "downsampled", "edsr", "espcn", "fsrcnn", "lapsrn"]
images = [img, img_small_resize, img_upscaled1, img_upscaled2, img_upscaled3, img_upscaled4]
plot_images(images, titles)